<h1 style="color:blue"> Lisamaterjal: verbiahelate tuvastamine. Andmete serialiseerimine </h1>

See lisamaterjal käsitleb üht uut keeletöötluse teemat ning üht "tehnilist" Pythoni teemat.
Keeletöötluse teemaks on _verbiahelate tuvastamine_ , mis võimaldab uurida tekstis olevaid väiteid (nt eristada jaatavaid väiteid eitavatest ja kindlaid väiteid võimalikest).
Tehniline teema on aga _serialiseerimine_ , mis võimaldab Pythoni mälus olevaid andmeid (sh objekte) otse faili salvestada ning pärast ka taastada.

## Verbiahelate tuvastamine

Kui lause eesmärgiks on mingi väite väljendamine, siis selle väite _faktilisust_ / _modaalsust_ mõjutab sageli see, milline on väidet väljendav _verbiahel_ . 
Näiteks, verbiahelaid uurides võib eristada jaatavaid väiteid eitavatest (nt "Peaminister **astub** homme tagasi" vs "Peaminister eile siiski tagasi **ei astunud**"), kindlaid väiteid võimalikest (nt "Ta **tuleb** homme" vs "Ta **võib tulla** homme") ning kindlaid tegevusi soovidest / plaanidest / üritamistest jms (nt "Juku **alustas** kodutööga" vs "Juku **kavatseb** kodutööga **alustada**", '"Juku **tahab** kodutööd **alustada**", "Juku kinnitusel **on võimalik** kodutööd **alustada**"). 
Verbiahelate tuvastaja leiabki verbidega seotud ühe- ja mitmesõnalisi üksuseid, millele toetudes saab üles ehitada lause _faktilisuse_ / _modaalsuse_ automaatse analüüsi.

Näide verbiahelate tuvastaja importimisest:

In [1]:
from estnltk.taggers import VerbChainDetector
vc_detector = VerbChainDetector()
vc_detector

VerbChainDetector(input_layers=('words', 'sentences', 'morph_analysis', 'clauses'), output_layer=verb_chains, output_attributes=('pattern', 'roots', 'word_ids', 'mood', 'polarity', 'tense', 'voice', 'remaining_verbs'), add_morph_attr=False, add_analysis_ids_attr=False, expand2ndTime=False, breakOnPunctuation=False, removeSingleAraEi=True)

Lihtsaimaks kasutusnäiteks on jaatavate ja eitavate verbiahelate tuvastamine:

In [2]:
from estnltk import Text

# Loome teksti
text = Text('Kas Juku alustas kodutööga? Minuteada ei alustanud.')
# Lisame verbiahelate tuvastamiseks vajalikud sisendkihid
text.tag_layer(['words', 'sentences', 'morph_analysis', 'clauses'])

# Tuvastame verbiahelad
vc_detector.tag( text )

# Väljastame verbiahelad (vastavad tekstid)
text.verb_chains

Layer(name='verb_chains', attributes=('pattern', 'roots', 'word_ids', 'mood', 'polarity', 'tense', 'voice', 'remaining_verbs'), spans=SL[EnvelopingSpan(['alustas'], [{'pattern': ['verb'], 'roots': ['alusta'], 'word_ids': [2], 'mood': 'indic', 'polarity': 'POS', 'tense': 'imperfect', 'voice': 'personal', 'remaining_verbs': False}]),
EnvelopingSpan(['ei', 'alustanud'], [{'pattern': ['ei', 'verb'], 'roots': ['ei', 'alusta'], 'word_ids': [6, 7], 'mood': 'indic', 'polarity': 'NEG', 'tense': 'imperfect', 'voice': 'personal', 'remaining_verbs': False}])])

_Ühe- vs mitmesõnalised ahelad._ Nagu eelmisest näitest võib näha, siis jaatav "verbiahel" võib koosneda ainult ühest sõnast. Seepärast võib nimi "verbiahel" esmapilgul tunduda veidi eksitav (mismoodi on üks verb ahel?!), aga kuna verbiahelate tuvastaja eesmärgiks on süstemaatiliselt leida üles kõik lause / osalause peaverbe¹ hõlmavad konstruktsioonid ja peaverbid ongi sageli ühesõnalised, siis tagastab ka tuvastaja nii mitmesõnalisi kui ka ühesõnalisi tekstiüksuseid;

 * ¹ _peaverbi_ all mõeldakse siin enamasti finiitverbi ehk pöördelist verbivormi. Aga on ka erandeid, nende kohta vt järgnevat loendit;

Konkreetsemalt katab verbiahelate tuvastaja järgmisi konstruktsioone:
   * Lause/osalause peaverbid:
       * Jaatavad verbid, v.a. _olema_ (nt "Pidevalt **uurivad** asjade seisu ka hollandlased");
       * Ühe- ja mitmesõnalised jaatavad _olema_ -konstruktsioonid (nt "Raha **on** alati vähe", "**Oleme** sellist kino ennegi **näinud**");
       * Eitusega peaverbid: _ei/ära/pole/ega + verb_ (nt "Helistasin korraks Carmenile, kuid ta **ei vastanud**");    
   * Peaverbide laiendused:
       * _verb + infiniitverb_ ahel, kus _infiniitverbi_ ja eelneva verbi vahel on rektsiooniseos, nt verb _kutsuma_ saab võtta _ma_ -verbidest argumente ("Kevadpäike **kutsub** mind **suusatama**") ning _püüdma_ saab võtta _da_ -verbidest argumente ("Aita **ei püüdnudki** Leenat **mõista**");
       * _verb + nom/adv + infiniitverb_ ahel, kus mitmesõnaline üksus _verb + nimisõna/adverb_ on rektsiooniseoses sellele järgneva infiniitverbiga; näiteks, verb _otsima_ saab moodustada mitmesõnalise üksuse sõnaga _võimalust_ ning see üksus saab võtta omakorda _da_ -verbe argumentideks ("Seepärast **otsisimegi** **võimalust** kusagilt mõned ilvesed **hankida**");

Kui rektsiooniseoste tuvastamisel ei teki mitmesusi (st ei tuvastata mitut erinevat viisi ühe ahela laiendamiseks), võib verbiahelate tuvastaja algoritm laiendada üht ahelat ka mitu korda ning moodustada üksjagu pika ahela, nt lausest "Minul oleks pidanud olema õigus ise endale külalisi kutsuda" leitakse ahel  **oleks** => **pidanud** => **olema** => **õigus** => **kutsuda**.

In [3]:
# Loome teksti ja lisame verbiahelate tuvastamiseks vajalikud kihid
text = Text('Juku tahtis kodutööd juba ammu alustada, aga ei saanud. Juku kinnitusel on siiski võimalik kodutööd alustada.')
text.tag_layer(['words', 'sentences', 'morph_analysis', 'clauses'])

# Tuvastame verbiahelad
vc_detector.tag( text )

# Väljastame verbiahelad (vastavad tekstid)
text.verb_chains

Layer(name='verb_chains', attributes=('pattern', 'roots', 'word_ids', 'mood', 'polarity', 'tense', 'voice', 'remaining_verbs'), spans=SL[EnvelopingSpan(['tahtis', 'alustada'], [{'pattern': ['verb', 'verb'], 'roots': ['taht', 'alusta'], 'word_ids': [1, 5], 'mood': 'indic', 'polarity': 'POS', 'tense': 'imperfect', 'voice': 'personal', 'remaining_verbs': False}]),
EnvelopingSpan(['ei', 'saanud'], [{'pattern': ['ei', 'verb'], 'roots': ['ei', 'saa'], 'word_ids': [8, 9], 'mood': 'indic', 'polarity': 'NEG', 'tense': 'imperfect', 'voice': 'personal', 'remaining_verbs': False}]),
EnvelopingSpan(['on', 'võimalik', 'alustada'], [{'pattern': ['ole', 'nom/adv', 'verb'], 'roots': ['ole', 'võimalik', 'alusta'], 'word_ids': [13, 15, 17], 'mood': 'indic', 'polarity': 'POS', 'tense': 'present', 'voice': 'personal', 'remaining_verbs': False}])])

Millist infot annavad edasi tuvastatud verbiahelate atribuudid?

* `text` -- verbiahela fraas. Kuna verbiahelate kiht on ümbriskiht sõnade kihi ümber, siis on siin sõnade järjend.
* `pattern` -- järjend ahela sõnade üldistusega; ahelat üldistav muster. Iga sõna kohta on toodud välja, kas sõna on: 'verb' (tegusõna, v.a 'olema' verbid), 'nom/adv' (käändsõna või adverb), 'ole' ('olema'-verb), 'pole' ('olema'-verbi eitus: pole & polnud jne), '&' (sidesõna: ja/ning/ega/või) või siis 'ega', 'ei' või 'ära';
* `roots` -- järjend ahela sõnade algvormidega (morf analüüsi `'root'` atribuudid);
* `word_ids` -- järjend ahela moodustavate sõnade indeksitega tekstis;
* `mood` -- ahela finiitverbi kõneviis. Võimalikud väärtused: `'indic'` (kindel kv), `'imper'` (käskiv ja möönev), `'condit'` (tingiv), `'quotat'` (kaudne),  või `'??'` (määramata);
* `polarity` -- kas verbiahel on jaatav (`'POS'`) või eitav (`'NEG'`)? Võimalik ka väärtus `'??'` (määramata);
* `tense` -- ahela finiitverbi/konstruktsiooni grammatiline aeg. Võimalikud väärtused sõltuvad kõneviisist. Kindlas kõneviisis: `'present'`, `'imperfect'`, `'perfect'`, `'pluperfect'`, käskivas kõneviisis:  `'present'` ning tingivas ja kaudses: `'present'` ja `'past'`. Lisaks võib jääda ka määramata (`'??'`);
* `voice` -- ahela finiitverbi tegumood. Võimalikud väärtused: `'personal'`, `'impersonal'`, `'??'`;
* `remaining_verbs` -- kahendmuutuja, mis näitab, kas osalausesse jäi veel nö lahtisi verbe, mis potentsiaalselt võiksid kuuluda ahelasse. Väärtus `True` annab märku sellest, et ahel võib olla mitte-täielikult eraldatud (st mitmesuse või puuduva rektsiooniinfo tõttu ei osanud algoritm kõiki verbe ahelasse panna / terviklikku ahelat moodustada);

**Sõnade järjekord ahelas. Tähenduse mõjutajad ja sisuverbid.** Atribuudi `roots` all olevad algvormid ei ole organiseeritud mitte nende lauses esinemise järjekorras, vaid järjekorras, mis peegeldab sõnadevahelisi seoseid: 
iga järgnev sõna ahelas on eelmise sõna süntaktiline alluv.
Sellise järjestamise tulemusena on ahela eesotsas _tähendust mõjutavad verbid_ , nt eituse abiverb või modaalverb, ning ahela lõpus _sisuverb_ ehk siis verb, mis annab edasi väite põhisisu.
Näited:

In [4]:
text = Text('Ja teda kutsuda ei proovitudki? Plaanis on lihtsalt hängida seal.')
text.tag_layer(['words', 'sentences', 'morph_analysis', 'clauses'])

# Tuvastame verbiahelad
vc_detector.tag( text )

# Väljastame verbiahelad
text.verb_chains[['text', 'roots', 'pattern']]

,text,roots,pattern
0,"['kutsuda', 'ei', 'proovitudki']","['ei', 'proovi', 'kutsu']","['ei', 'verb', 'verb']"
1,"['Plaanis', 'on', 'hängida']","['ole', 'plaan', 'hängi']","['ole', 'nom/adv', 'verb']"


Nagu näha, siis atribuut `text` annab edasi sõnade tekstis olevat järjestust (sama kehtib ka atribuutide `start` ja `end` kohta), samas kui järjendites `roots` ja `pattern` on järjestus vastavalt sõnade vahelistele seostele. 

Rohkem infot verbiahelate tuvastaja kohta leiab [EstNLTK abimaterjalist](https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/X_miscellaneous/01_verb_chain_detector.ipynb).

---

## Serialiseerimine ( _Pickle_ )

Tekstilised formaadid nagu XML ja JSON on inimesele suhteliselt hästi loetavad, kuid nende lugemine/kirjutamine tähendab arvuti jaoks enamasti lisatööd: andmeid tuleb teisendada mälus oleva kuju ja tekstilise kuju vahel. 
Kui lood ise uue Pythoni klassi, siis ei saa selle objekte kohe JSON / XML kujul salvestada, vaid pead looma ka konverteri, mis teisendab objekte vastavatesse andmekujudesse (ja ka tagasi).

Pythoni andmestruktuuride ja objektide otse-salvestamiseks leidub ka alternatiivne ja kiirem viis, mida nimetatakse _serialiseerimiseks_. 
Sisuliselt tähendab serialiseerimine, et salvestamisel teisendatakse mälus olevad andmed binaarsele kujule ning failist laadimisel muudetakse need tagasi Pythoni mälus olevateks andmeteks.
Kuna andmete formaati sealjuures ei muudeta, on protsess enamasti suhteliselt kiire.
Samuti ei pea ise tegelema konverteri kirjutamisega: enamikku andmestruktuure, funktsioone, klasse kui ka klasside isendeid / objekte on võimalik serialiseerida ilma, et peaks neid kuidagi teisendama.

Pythonis vastutab serialiseerimise eest teek [_pickle_](https://docs.python.org/3.9/library/pickle.html). Nagu teiste andmeformaatidegi puhul, on ka serialiseerimise tulemused platvormist sõltumatud ehk serialiseerimise abil tekitatud fail peaks olema suvalisel (Pythoniga) masinal loetav. Binaarse kuju miinuseks on aga kahjuks see, et failis olevad andmed pole enam inimsilmale loetavad.

Näitena serialiseerime _rows_ muutuja.

In [5]:
rows = [{'a':5, 'b':3}, {'c':4, 'd':1 }]

import pickle
with open('serialized_ppl.pkl','wb') as fout:
    pickle.dump(rows, fout)

Paneme tähele, et fail tuleb avada nii lugemiseks kui kirjutamiseks _binary mode_'is (`'wb'` ja `'rb'`).

Võite iseseisvalt uurida, milline on tekitatud faili `'serialized_ppl.pkl'` sisu. (Eriti loetav see pole.)

Funktsiooni `pickle.load` abil saame serialiseeritud sisu jälle Pythonisse tagasi lugeda:

In [6]:
with open('serialized_ppl.pkl','rb') as fin:
    new_rows = pickle.load(fin)
    print(new_rows)

[{'a': 5, 'b': 3}, {'c': 4, 'd': 1}]


Vaikimisi saab _pickle_ hakkama paljude standardteekide klasside / objektide serialiseerimisega ning ka endakirjutatud teekidega, mis kasutavad standardteegi objekte.

### EstNLTK `Text` objektide serialiseerimisest

Ka EstNLTK `Text` objektide salvestamiseks/laadimiseks on serialiseerimine üsna kiire viis. Proovime seda ühe näite varal:

In [7]:
# Loome serialiseeritava teksti
from estnltk import Text
text = Text('Küll homme armastab see, kes ei armastanud eile.')
# Lisame märgendused
text = text.tag_layer(['clauses'])
# Tuvastame verbiahelad
vc_detector.tag( text )

import pickle
with open('test_serialized_text.pkl','wb') as fout:
     pickle.dump(text, fout)

Laeme teksti sisse ja uurime, kas märgendused säilisid:

In [8]:
with open('test_serialized_text.pkl','rb') as fin:
    loaded_text = pickle.load(fin)

display(loaded_text)
display(loaded_text.clauses)
display(loaded_text.verb_chains)

Text(text='Küll homme armastab see, kes ei armastanud eile.')

Layer(name='clauses', attributes=('clause_type',), spans=SL[EnvelopingSpan(['Küll', 'homme', 'armastab', 'see', ','], [{'clause_type': 'regular'}]),
EnvelopingSpan(['kes', 'ei', 'armastanud', 'eile', '.'], [{'clause_type': 'regular'}])])

Layer(name='verb_chains', attributes=('pattern', 'roots', 'word_ids', 'mood', 'polarity', 'tense', 'voice', 'remaining_verbs'), spans=SL[EnvelopingSpan(['armastab'], [{'pattern': ['verb'], 'roots': ['armasta'], 'word_ids': [2], 'mood': 'indic', 'polarity': 'POS', 'tense': 'present', 'voice': 'personal', 'remaining_verbs': False}]),
EnvelopingSpan(['ei', 'armastanud'], [{'pattern': ['ei', 'verb'], 'roots': ['ei', 'armasta'], 'word_ids': [6, 7], 'mood': 'indic', 'polarity': 'NEG', 'tense': 'imperfect', 'voice': 'personal', 'remaining_verbs': False}])])

Mis läheb serialiseerimisel `Text`-iga kaasa? Analoogselt `Text`-ide salvestamisega JSON formaadis pannakse kaasa algne tekst, teksti meta-andmed ja märgenduskihid. Välja jääb info selle kohta, milliste märgendajatega kihid tekitati ning millised olid märgendajate konfiguratsioonid.

---